In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 8)

from scipy import stats
import collections

import warnings
# warnings.filterwarnings('ignore')
from matplotlib.backends.backend_pdf import PdfPages

from pathlib import Path

import db_queries as db
import vivarium_helpers.id_helper as idh
import gbd_mapping
from vivarium import Artifact

# Add the repo directory vivarium_research_ciff_sam/ to sys.path
import os, sys
repo_path = os.path.abspath('../..')
sys.path.append(repo_path)
# Assumes vivarium_research_ciff_sam/ is in sys.path
import model_validation.vivarium_transformed_output as vto
# import model_validation.vivarium_raw_output as vro
import model_validation.vivarium_output_processing as vop
import model_validation.ciff_sam_results as csr

!pwd
!whoami
!date

/ihme/homes/ndbs/vivarium_research_ciff_sam/model_validation/model4
ndbs
Mon Sep 27 15:24:54 PDT 2021


In [2]:
%load_ext autoreload
%autoreload 2

# Load data and compute total person time

In [3]:
data = csr.VivariumResults.cleaned_from_model_spec(4.1)
data.table_names()

['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'stunting_state_person_time',
 'population',
 'ylls',
 'ylds',
 'cause_state_person_time',
 'cause_transition_count']

In [4]:
data.compute_total_person_time()
data.table_names()

['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'stunting_state_person_time',
 'population',
 'ylls',
 'ylds',
 'cause_state_person_time',
 'cause_transition_count',
 'person_time']

# Compute wasting prevalence in each category by scenario and summarize over draws

In [5]:
query_string = "year>'2022' and age!='all_ages'" # Filter out pre-intervention years
wasting_prevalence = vop.ratio(
    data.wasting_state_person_time.query(query_string),
    data.person_time.query(query_string),
    strata=[], # Stratifying by nothing groups by scenario and draw
    numerator_broadcast='wasting_state', # Compute prevalence of all categories simultaneously
    multiplier=100, # Convert units to percent
)
wasting_prevalence

,input_draw,scenario,wasting_state,value,numerator_measure,denominator_measure,multiplier
0,29,baseline,mild_child_wasting,21.471650,state_person_time,person_time,100
1,29,baseline,moderate_acute_malnutrition,8.004558,state_person_time,person_time,100
2,29,baseline,severe_acute_malnutrition,1.947800,state_person_time,person_time,100
3,29,baseline,susceptible_to_child_wasting,68.575993,state_person_time,person_time,100
...,...,...,...,...,...,...,...
140,946,wasting_treatment,mild_child_wasting,22.320811,state_person_time,person_time,100
141,946,wasting_treatment,moderate_acute_malnutrition,6.996664,state_person_time,person_time,100
142,946,wasting_treatment,severe_acute_malnutrition,1.611424,state_person_time,person_time,100
143,946,wasting_treatment,susceptible_to_child_wasting,69.071101,state_person_time,person_time,100


In [7]:
vop.describe(wasting_prevalence)

count  \
denominator_measure multiplier numerator_measure scenario          wasting_state                         
person_time         100        state_person_time baseline          mild_child_wasting             12.0   
                                                                   moderate_acute_malnutrition    12.0   
                                                                   severe_acute_malnutrition      12.0   
                                                                   susceptible_to_child_wasting   12.0   
...                                                                                                ...   
                                                 wasting_treatment mild_child_wasting             12.0   
                                                                   moderate_acute_malnutrition    12.0   
                                                                   severe_acute_malnutrition      12.0   
                                                                   susceptible_to_child_wasting   12.0   

                                                                                                      mean  \
denominator_measure multiplier numerator_measure scenario          wasting_state                             
person_time         100        state_person_time baseline          mild_child_wasting            21.516838   
                                                                   moderate_acute_malnutrition    7.955648   
                                                                   severe_acute_malnutrition      1.887599   
                                                                   susceptible_to_child_wasting  68.639915   
...                                                                                                    ...   
                                                 wasting_treatment mild_child_wasting            22.409269   
                                                                   moderate_acute_malnutrition    6.907774   
                                                                   severe_acute_malnutrition      1.601461   
                                                                   susceptible_to_child_wasting  69.081496   

                                                                                                      std  \
denominator_measure multiplier numerator_measure scenario          wasting_state                            
person_time         100        state_person_time baseline          mild_child_wasting            0.127737   
                                                                   moderate_acute_malnutrition   0.083820   
                                                                   severe_acute_malnutrition     0.099181   
                                                                   susceptible_to_child_wasting  0.165686   
...                                                                                                   ...   
                                                 wasting_treatment mild_child_wasting            0.167146   
                                                                   moderate_acute_malnutrition   0.174895   
                                                                   severe_acute_malnutrition     0.069125   
                                                                   susceptible_to_child_wasting  0.169451   

                                                                                                       min  \
denominator_measure multiplier numerator_measure scenario          wasting_state                             
person_time         100        state_person_time baseline          mild_child_wasting            21.251070   
                                                                   moderate_acute_malnutrition    7.831141   
                                                                   severe_acute

In [6]:
vop.describe(wasting_prevalence)['mean'].unstack().round(2)

wasting_state                                                       mild_child_wasting  \
denominator_measure multiplier numerator_measure scenario                                
person_time         100        state_person_time baseline                        21.52   
                                                 sqlns                           23.02   
                                                 wasting_treatment               22.41   

wasting_state                                                       moderate_acute_malnutrition  \
denominator_measure multiplier numerator_measure scenario                                         
person_time         100        state_person_time baseline                                  7.96   
                                                 sqlns                                     6.15   
                                                 wasting_treatment                         6.91   

wasting_state                                                       severe_acute_malnutrition  \
denominator_measure multiplier numerator_measure scenario                                       
person_time         100        state_person_time baseline                                1.89   
                                                 sqlns                                   1.46   
                                                 wasting_treatment                       1.60   

wasting_state                                                       susceptible_to_child_wasting  
denominator_measure multiplier numerator_measure scenario                                         
person_time         100        state_person_time baseline                                  68.64  
                                                 sqlns                                     69.37  
                                                 wasting_treatment                         69.08

# Put scenarios and wasting categories in the order I want

In [10]:
ordered_scenarios = ['baseline', 'wasting_treatment', 'sqlns']
ordered_wasting_states = [
    'severe_acute_malnutrition',
    'moderate_acute_malnutrition',
    'mild_child_wasting',
    'susceptible_to_child_wasting',
]
wasting_prevalence['scenario'] = pd.Categorical(
    wasting_prevalence['scenario'], categories=ordered_scenarios, ordered=True)
wasting_prevalence['wasting_state'] = pd.Categorical(
    wasting_prevalence['wasting_state'], categories=ordered_wasting_states, ordered=True)
wasting_prevalence

,input_draw,scenario,wasting_state,value,numerator_measure,denominator_measure,multiplier
0,29,baseline,mild_child_wasting,21.471650,state_person_time,person_time,100
1,29,baseline,moderate_acute_malnutrition,8.004558,state_person_time,person_time,100
2,29,baseline,severe_acute_malnutrition,1.947800,state_person_time,person_time,100
3,29,baseline,susceptible_to_child_wasting,68.575993,state_person_time,person_time,100
...,...,...,...,...,...,...,...
140,946,wasting_treatment,mild_child_wasting,22.320811,state_person_time,person_time,100
141,946,wasting_treatment,moderate_acute_malnutrition,6.996664,state_person_time,person_time,100
142,946,wasting_treatment,severe_acute_malnutrition,1.611424,state_person_time,person_time,100
143,946,wasting_treatment,susceptible_to_child_wasting,69.071101,state_person_time,person_time,100


In [12]:
vop.describe(wasting_prevalence)['mean'].unstack().round(1)

wasting_state                                                       severe_acute_malnutrition  \
denominator_measure multiplier numerator_measure scenario                                       
person_time         100        state_person_time baseline                                 1.9   
                                                 wasting_treatment                        1.6   
                                                 sqlns                                    1.5   

wasting_state                                                       moderate_acute_malnutrition  \
denominator_measure multiplier numerator_measure scenario                                         
person_time         100        state_person_time baseline                                   8.0   
                                                 wasting_treatment                          6.9   
                                                 sqlns                                      6.1   

wasting_state                                                       mild_child_wasting  \
denominator_measure multiplier numerator_measure scenario                                
person_time         100        state_person_time baseline                         21.5   
                                                 wasting_treatment                22.4   
                                                 sqlns                            23.0   

wasting_state                                                       susceptible_to_child_wasting  
denominator_measure multiplier numerator_measure scenario                                         
person_time         100        state_person_time baseline                                   68.6  
                                                 wasting_treatment                          69.1  
                                                 sqlns                                      69.4

# Get the 95% UI values

In [13]:
vop.describe(wasting_prevalence)['2.5%'].unstack().round(1)

wasting_state                                                       severe_acute_malnutrition  \
denominator_measure multiplier numerator_measure scenario                                       
person_time         100        state_person_time baseline                                 1.8   
                                                 wasting_treatment                        1.5   
                                                 sqlns                                    1.4   

wasting_state                                                       moderate_acute_malnutrition  \
denominator_measure multiplier numerator_measure scenario                                         
person_time         100        state_person_time baseline                                   7.8   
                                                 wasting_treatment                          6.6   
                                                 sqlns                                      5.9   

wasting_state                                                       mild_child_wasting  \
denominator_measure multiplier numerator_measure scenario                                
person_time         100        state_person_time baseline                         21.3   
                                                 wasting_treatment                22.2   
                                                 sqlns                            22.7   

wasting_state                                                       susceptible_to_child_wasting  
denominator_measure multiplier numerator_measure scenario                                         
person_time         100        state_person_time baseline                                   68.4  
                                                 wasting_treatment                          68.9  
                                                 sqlns                                      69.2

In [14]:
vop.describe(wasting_prevalence)['97.5%'].unstack().round(1)

wasting_state                                                       severe_acute_malnutrition  \
denominator_measure multiplier numerator_measure scenario                                       
person_time         100        state_person_time baseline                                 2.1   
                                                 wasting_treatment                        1.7   
                                                 sqlns                                    1.5   

wasting_state                                                       moderate_acute_malnutrition  \
denominator_measure multiplier numerator_measure scenario                                         
person_time         100        state_person_time baseline                                   8.1   
                                                 wasting_treatment                          7.2   
                                                 sqlns                                      6.4   

wasting_state                                                       mild_child_wasting  \
denominator_measure multiplier numerator_measure scenario                                
person_time         100        state_person_time baseline                         21.7   
                                                 wasting_treatment                22.7   
                                                 sqlns                            23.3   

wasting_state                                                       susceptible_to_child_wasting  
denominator_measure multiplier numerator_measure scenario                                         
person_time         100        state_person_time baseline                                   68.9  
                                                 wasting_treatment                          69.4  
                                                 sqlns                                      69.6